---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [6]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data.head(10)

,target,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
5,1,FreeMsg Hey there darling it's been 3 week's n...
6,0,Even my brother is not like to speak with me. ...
7,0,As per your request 'Melle Melle (Oru Minnamin...
8,1,WINNER!! As a valued network customer you have...
9,1,Had your mobile 11 months or more? U R entitle...


In [ ]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [ ]:
def answer_one():
    not_spam, spam = spam_data['target'].value_counts()
    
    return spam/(spam+not_spam) * 100

In [ ]:
# answer_one()

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    vect = CountVectorizer().fit(X_train)
    feature = vect.get_feature_names()
    
    return max(feature, key=len)

In [ ]:
# answer_two()

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    vect = CountVectorizer().fit(X_train)
    X_vect = vect.transform(X_train)
    mnb = MultinomialNB(alpha=0.1).fit(X_vect, y_train)
    pred = mnb.predict(vect.transform(X_test))
    return roc_auc_score(y_test, pred)

In [ ]:
# answer_three()

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():
    vect = TfidfVectorizer().fit(X_train)
    X_vect = vect.transform(X_train)
    
    sorted_value = sorted(X_vect.max(0).toarray()[0])
    sorted_index = X_vect.max(0).toarray()[0].argsort()
    features = np.array(vect.get_feature_names())
    large_series = pd.Series(sorted_value[-20:],index=features[sorted_index[-20:]])
    small_series = pd.Series(sorted_value[:20],index=features[sorted_index[:20]])
    return small_series, large_series

In [ ]:
# answer_four()

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [ ]:
def answer_five():
    vect = TfidfVectorizer(min_df=3).fit(X_train)
    X_vect = vect.transform(X_train)
    
    mnb = MultinomialNB(alpha=0.1).fit(X_vect, y_train)
    pred = mnb.predict(vect.transform(X_test))
    
    return roc_auc_score(y_test, pred)

In [ ]:
# answer_five()

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [ ]:
def answer_six():
    temp = spam_data.copy()
    temp['text'] = temp['text'].apply(len)
    temp = temp.groupby('target').mean()
    
    return temp.loc[0, 'text'], temp.loc[1, 'text']

In [ ]:
# answer_six()

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [ ]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [ ]:
from sklearn.svm import SVC

def answer_seven():
    vect = TfidfVectorizer(min_df=5).fit(X_train)
    X_vect = vect.transform(X_train)
    
    X_final = add_feature(X_vect, X_train.apply(len))
    
    svm = SVC(C=10000).fit(X_final, y_train)
    pred = svm.predict(add_feature(vect.transform(X_test), X_test.apply(len)))
    
    return roc_auc_score(y_test, pred)

In [ ]:
# answer_seven()

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [ ]:
def answer_eight():
    temp = spam_data.copy()
    temp['text'] = temp['text'].apply(lambda x: sum([1 for i in list(x) if i.isdigit()]))
    temp = temp.groupby('target').mean()
    
    return temp.loc[0, 'text'], temp.loc[1, 'text']

In [ ]:
# answer_eight()

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [ ]:
from sklearn.linear_model import LogisticRegression

def answer_nine():
    vect = TfidfVectorizer(min_df=5, ngram_range=(1,3)).fit(X_train)
    
    def add_2features(X):
        X_vect = vect.transform(X)
        return add_feature(
            add_feature(X_vect, X.apply(len)),
            X.apply(lambda x: sum([1 for i in list(x) if i.isdigit()]))
        )
    
    X_final = add_2features(X_train)
    
    reg = LogisticRegression(C=100).fit(X_final, y_train)
    pred = reg.predict(add_2features(X_test))
    
    return roc_auc_score(y_test, pred)

In [ ]:
# answer_nine()

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [ ]:
def answer_ten():
    from re import findall
    temp = spam_data.copy()
    temp['text'] = temp['text'].apply(lambda x: len(findall(r'\W', x)))
    temp = temp.groupby('target').mean()
    
    return temp.loc[0, 'text'], temp.loc[1, 'text']

In [ ]:
# answer_ten()

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [ ]:
def answer_eleven():
    from re import findall
    vect = CountVectorizer(min_df=5, ngram_range=(2,5), analyzer='char_wb').fit(X_train)
    
    def add_3features(X):
        X_vect = vect.transform(X)
        return add_feature(add_feature(add_feature(X_vect, X.apply(len)),
                                       X.apply(lambda x: len(findall(r'\d', x)))),
                           X.apply(lambda x: len(findall(r'\W', x))))
    
    X_final = add_3features(X_train)
    
    reg = LogisticRegression(C=100).fit(X_final, y_train)
    pred = reg.predict(add_3features(X_test))
    
    features = np.array(vect.get_feature_names() + ['length_of_doc', 'digit_count', 'non_word_char_count'])
    sorted_index = reg.coef_[0].argsort()
    
    small_coef = features[sorted_index[:10]]
    large_coef = features[sorted_index[-10:]]
    
    return roc_auc_score(y_test, pred), small_coef, large_coef[::-1]

In [ ]:
# answer_eleven()